
# Coding Conventions for COFFE developers


## Programming Language and Library
Use Python 3 syntax. 
Feel free to use python's libraries, like
- numpy
- pytest
- shutil
- tempfile
- ...

## Git
It is highly recommended to use git for version control.
- Commit regularly.
- Merge the reomte develop branch into your branch regularly. This will help other developers to use your stuff - and enable you to use theirs.
- Before you commit to the remote develop branch, make sure that you have pulled and merged the current version. Make sure that all unit tests are running seemlessly.
- Use meaningful commit messages.

## Program Structure

### COFFE Main Directory

| Directory | Content |
|-----------|---------|
| coffe     | Source code |
| docs      | Documentation (like this notebook) |
| share     | Files that are distributed with the software (no source code). These include: Steering scripts, Bash scripts for cluster submission, structure files, etc.|
| examples  | Tutorials to help others use your code. |
| tests     | Unit tests |

### Source Directory (coffe)

| Subdirectory | Content |
|-----------|---------|
| core      | core functionality, should remain more or less unchanged |
| gmx     | Gromacs simulation classes and functions |
| amb     | Amber simulation classes and functions |
| ...     | to be continued |




### Classes and Functions

Simplistic programming style!

Use classes only for computationally expensive tasks (usually simulations), and functions elsewhere.

Using classes for expensive tasks has the following advantages:
- They can be stored and loaded.
- Input can be checked on the head node of a cluster, while a computing node does the actual work. This can avoid runtime errors after submission.

Simulation classes and their use are described in a separate notebook.

### Unit Testing

The test directory has the same structure as the source directory.
Write unit tests for each function or class.
Use py.test for testing.


## Indentation

Use four (4) spaces for indentation. **Do not use tabs.**
Preferrably, edit your editor preferences so that each tab you insert gets replaced by 4 spaces automatically.

## Naming Conventions

Names should be descriptive, i.e. they should resemble the functionality of a piece of code accurately. When writing your own functions and classes, please stick to the following conventions:

1) Function names start with a lowercase letter, words are seperated by underscores, e.g. 

```
def gmx_mkbox_homogeneous():
    pass
```

2) Class names are CamelCase and start with an uppercase letter, e.g.

```
class GmxEquilibrationNVT:
    pass
```

3) Functions and classes that are specific to a simulation engine should start with one of the following abbreviations


| Program | Abbreviation |
|---------|--------------|
|Gromacs|gmx|
|Amber|amb|
|CHARMM|chm|
|OpenMM|omm|
|Psi4|psi|
|GAMESS|gam|
|NAMD|nmd|
|QChem|qch|

4) Local variables and function parameters are lowercase and seperated by underscores, e.g.

```
def gmx_mkbox_homogeneous(input_structure, input_topology, nmol, box_size):
    pass
```




## Comments
1) Use comments wisely. Do not overload your code with comments, but do not use them too sparingly, either. **If the code speaks for itself, comments are not required.**

2) Use python's docstrings to comment modules, functions and classes. Describe accurately

-  what a function or class does,
-  which arguments it takes,
-  what it returns, and
-  what exceptions could be raised.

3) Use Google-style documentation [(you can make this the default in PyCharm)](https://www.jetbrains.com/help/pycharm/python-integrated-tools.html).


### A few tricks Google-style Documentation:
Python uses Sphinx (with nbsphinx sphinx_click) to automatically generate the API's documentation.
To rebuild the documentation, you can use the python script 
*coffe/docs/update_docs.*

Here are a few tricks and hints:

#### a) Reference other functions/classes/methods:
To reference another function's documentation, use
```
:func:`~coffe.core.decorators.args_from_configfile` 
```
To reference classes, methods, and modules, use `:class:`, `:meth:`, and `:mod:`, respectively.




#### b) Careful with decorators:
If you write decorators, use functools.wraps, as in coffe.core.decorators.args_from_configfile. Otherwise, they hide the documentation.



### Examples:
##### A bad (useless) comment:
```
# initialize i
i = 0
```

#### Good commenting style:


In [3]:
def gmx_mkbox_homogeneous(substance, n_mols, box_size, ff_dir=None, gmx_ff=None,
                          create_itp=True, include_topology=None, work_dir=".",
                          substance_name="substance", box_name="box"):
    """Make a box that contains molecules of a single species.

    Args:
        substance(str): Input structure file containing a single molecule.
        n_mols(int): Number of molecules.
        box_size(float or tuple of 3 floats): Box size in nm
        ff_dir(str): (Optional) Forcefield directory.
        gmx_ff(str): (Optional) Forcefield name of Gromacs' built-in force field.
        create_itp(bool): Flag to call pdb2gmx, if include_topology is not given (default: True).
        include_topology(str): (Optional) Include topology.
        work_dir(str): Working directory (default=".").
        substance_name(str): Name of the substance (default="substance").
        box_name(str): Name for the system (default="box").

    Returns:
        Tuple containing two filenames

            - structure (str): Structure file.
            - topology(str): Topology file.

    Raises:
        AssertionError: if input files or directories do not exist
        coffe.gmx.util.GromacsError: if something goes wrong with the gromacs command

    """
    # Create work dir and logger
    local_variables = locals()
    _work_dir, coffe_dir, logger = filesys.prepare_coffe_work_dir(work_dir)
    logger.info("Creating a homogeneous box with {}.".format(local_variables))

    # Check input
    try:
        _substance = filesys.make_abspath(substance, _work_dir)
        assert isinstance(n_mols, int) and n_mols > 0, "n_mols must be integer > 0"
        if include_topology is not None:
            _include_topology = filesys.make_abspath(include_topology, _work_dir)
        _ff_dir = ff_dir
        if ff_dir is not None:
            _ff_dir = filesys.make_abspath(ff_dir, _work_dir)
    except AssertionError as e:
        logger.exception(e)
        raise e

    # Create itp
    itp_list = [include_topology]
    if include_topology is None:
        if create_itp:
            itp = os.path.join(_work_dir, "include_topology.itp")
            itp, ff_itp = gmxutil.gmx_pdb2itp(_substance, itp, _ff_dir, gmx_ff, _work_dir)
            gmxutil.rename_substance_in_itp(itp, substance_name)
            itp_list = [itp]
        else:
            itp_list = []
    else:
        ff_itp = os.path.join("{}.ff".format(gmx_ff),"forcefield.itp")

    # Create box
    structure = filesys.make_abspath("conf.gro", _work_dir, check_exists=False)
    gmxutil.gmx_insert_n_molecules(box_size, _substance, n_mols, structure, _work_dir)

    # Create topology
    topology = gmxutil.gmx_make_top([substance_name], [n_mols], itp_list, os.path.dirname(ff_itp), _work_dir, box_name)
    logger.info("Homogeneous system created successfully.")
    logger.info(".... Structure file: {}".format(structure))
    logger.info(".... Topology file: {}".format(topology))
    return os.path.abspath(structure), os.path.abspath(topology)


 ## Assertions

Assertions are very useful to detect run-time errors. This can save a lot of time when debugging a piece of code. Check the pre- and post-conditions of a function explicitly, as in the following example:

In [4]:

def gmx_insert_n_molecules(initial_box, input_structure, nmol, final_box, log_file):
    """
     [ some docstring ]
    """
    
    assert os.path.isfile(initial_box),\
            "initial box does not exist ({})".format(initial_box)
    assert os.path.isfile(input_structure),\
            "input structure does not exist ({})".format(input_structure)
    assert isinstance(nmol, int), "nmol has to be an integer"
    
    
    # etc. etc. 
    
    return final_box


## Logging and the .coffe Directory

Each coffe function/class that calls subcommands, like gmx ..., or performs expensive tasks, should output detailed logging info and store the stdout/stderr of the subcommands. 
To facilitate this documentation, you should equip these functions/classes with a ```work_dir='.'``` argument and use the function ```prepare_coffe_work_dir``` from ```coffe.core.filesys```.

You will see the command

In [ ]:
_work_dir, coffe_dir, logger = prepare_coffe_work_dir(work_dir)

in many places in the code.

The returned ```_work_dir``` represents the original work_dir as an absolute path (with expanded environment variables).
** All relative paths are interpreted relative to the _work_dir!**
The returned ```coffe_dir``` is a hidden directory *.coffe* that is created in the work_dir.
It contains a log file log.txt and can be used to store temporary data and subcommand output.

The returned ```logger``` is an instance of python's ```logging.Logger``` and can be used to write output to the log file and console.
It provides different log levels.

Example usage:

In [ ]:
logger.debug("Some information that is purely for debugging.")
logger.info("Some information output for the log files.")
logger.warning("Warnings are written to the log file and shell.")
logger.error("Errors are also written to the shell.")
logger.exception(e) # Log exceptions in a try: ... except: block

To log exceptions, you have to call logger.exception in the except block.

In [ ]:
try:
    assert i==2
    assert isinstance(a,str), "a must be a string."
    some_risky_function(i,a)
except Exception as e:      # (You can also specify the type of exception here)
    logger.exception(e)     # logging output
    raise e                 # raise exception anyway

## Python 2/3 Compatibility

We should try to keep the code compatible with Python 2 and 3.
This means, that the py.test unit tests should be regularly run with different versions of python (this can be done by using tox, or by manually switching between python versions).

Compatibility with python 2 and 3 is enforced mainly in three places:
1. By adding the ```from __future__ import ...``` stuff at the beginning of each module.
2. By the six module. Six is a python modules that resolves inconsistencies between different Python versions. An example: the configparser module was renamed in Python 3. To resolve this inconsistency import the module as follows: ```from six.moves import configparser```.
3. Some inconsistencies are not resolved in six. If you want to implement your own fixes, please put it into *coffe/core/compat.py* and take a look at the examples there.
